<a href="https://colab.research.google.com/github/Utkarshkarki/RAG/blob/main/RAG_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
if "COLAB_GPU" in os.environ:
  print("[INFO] Running in Google Colab,installing requirements.")
  !pip install PyMuPDF
  !pip install tqdm
  !pip install accelerate
  !pip install bitsandbytes
  !pip install flash-attn --no-build-isolation

[INFO] Running in Google Colab,installing requirements.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 38.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [ ]:
!pip uninstall -y torch torchvision torchaudio transformers sentence-transformers
!pip install torch toechvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -U transformers sentence-transformers

Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126
Found existing installation: transformers 4.56.1
Uninstalling transformers-4.56.1:
  Successfully uninstalled transformers-4.56.1
Found existing installation: sentence-transformers 5.1.0
Uninstalling sentence-transformers-5.1.0:
  Successfully uninstalled sentence-transformers-5.1.0
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 1.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement toechvision (from versions: none)
ERROR: No matching distribution found for toechvision
     ━━━━━━━━━━━━

In [ ]:
import os
import requests

pdf_path ="human-nutrition-text.pdf"

if not os.path.exists(pdf_path):
  print("File doesn't exist, downloading....")

  #The URL of the pdf you want to download
  url=

  filename = pdf_path

  response  = requests.get(url)

  if response.status_code == 200:
     with open(filename,"wb") as file:
        file.write(response.content)
      print(f"The file has been downloaded and saved as {filename}")
  else:
    print(f"Failed to download the file. Status code:{response.status_code}")
else:
  print{f"File {pdf_path} exists."}


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 19)

In [ ]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
  """Perform minor formatting on text.""""
   cleaned_text = text.replace("\n"," ").strip()

   return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
  doc  = fitz.open(pdf_path)
  pages_and_texts = []
  for page_number , page in tqdm(enumerate(doc)):
    text = page.get_text()
    text = text_formatter(text)
    pages_and_texts.append({"page_number": page_number - 41,"page_char_count": len(text),
                            "page_word_count": len(text.split(" ")),
                            "page_sentence_count_raw": len(text.split(". ")),
                            "page_token_count": len(text) / 4,
                            "text": text})
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]


In [ ]:
import random

random.sample(pages_and_texts, k=3)

In [ ]:
import pandas as pd
df = pd.DataFrame(pages_and_texts)
df.head()

In [ ]:
df.describe().round(2)

In [ ]:
from spacy.lang.en import English
nlp=English()
nlp.add_pipe("sentencizer")
#create a document instance as an example
doc=nlp("This is a sentence.This another sentence")
assert len(list(doc.sents))==2

#Access the sentences of the document
list(doc.sents)

In [ ]:
for item in tqdm(pages_and_texts):
  item["sentences"]=list(nlp(item["text"]).sents)

  #Make sure all sentences are strings
  item["sentences"]=[str(sentence) for sentence in item["sentences"]]
  #count the sentences
  item["page_sentences_count_spacy"]=len(item["sentences"])

In [ ]:
#Inspect an example
random.sample(pages_and_texts,k=1)

In [ ]:
df=pd.dataFrame(pages_and_texts)
df.describe().round(2)

In [ ]:
#Define split size to turn groups of sentences into chunks
num_sentence_chunk_size=10

#create a function that recursively splits a list into desired sizes
def split_list(input_list: list,
               slice_size: int)-> list[list[str]]:
      """
      Splits the input_list into sublists of size slice_size (or as close as possible).
      For example, a list of 17 sentences would be split into two lists of[[10],[7]]
      """
      return [input_list[i:i + slice_size] for i in range(0,len(input_list),slice_size)]

   #Loop through pages and texts and split sentences into chunks

   for item in tqdm(pages_and_texts):
      item["sentence_chunks"]=split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
      item["num_chunks"]=len(item["sentence_chunks"])




In [ ]:
random.sample(pages_and_texts,k=1)

In [ ]:
#create a DataFrame to get stats
df=pd.dataFrame(pages_and_texts)
df.describe().round(2)

In [ ]:
#Define split size to turn groups of sentences into chunks
num_sentence_chunk_size=10

#Create a function that recursively splits a list into desired sizes
def split_list(input_list: list,
               slice_size: int)-> list[list[str]]:

    """
    Splits the input input_list into sublists of size slice_size(or as close as possible).

    for example, a list of 17 sentences would be split into two lists of [[10],[7]]
    """
    return [input_list[i:i + slice_size] for i in range(0,len(input_list),slice_size)]

#Loop through pages and texts and split sentences into chunks
for item in tqdm(pages_and_texts):
  item["sentence_chunks"]=split_list(input_list=item["sentences"],
                                     slice_size=num_sentence_chunk_size)
  item["num_chunks"]=len(item["sentence_chunks"])

In [ ]:
#Sample an example from the group (note: many samples have only 1 chunk as they have <=10 sentences total)

random.sample(pages_and_texts,k=1)

In [ ]:
#Create a dataframe to get stats
df=pd.dataFrame(pages_and_texts)
df.describe().round(2)

In [ ]:
import re
#Split each chunk into its own item
pages_and_chunks=[]
for item in tqdm(pages_and_texts):
  for sentence_chunk in item["sentence_chunk"]:
    chunk_dict={ }
    chunk_dict["page_number"]=item["page_number"]

    #join the sentences together into a paragraph-like structure, aka a chunk (so they are a single)
    joined_sentence_chunk="".join(sentence_chunk).replace("  "," ").strip()
    joined_sentence_chunk=re.sub(r'\.([A-Z])',r'.\1',joined_sentence_chunk)
    chunk_dict["sentence_chunk"] = joined_sentence_chunk

    #Get stats about the chunk
    chunk_dict["chunk_char_count"]=len(joined_sentence_chunk)
    chunk_dict["chunk_word_count"]=len([word for word in joined_sentence_chunk.split(" ")])
    chunk_dict["chunk_token_count"]=len(joined_sentence_chunk) / 4 #1 token =~4 characters

    pages_and_chunks.append(chunk_dict)
#How many chunks do we have?
len(pages_and_chunks)



In [ ]:
random.sample(pages_and_chunks, k=1)

In [ ]:
#Get stats about our chunks
df=pd.DataFrame(pages_and_chunks)
df.describe().round(2)

In [ ]:
#show random chunks with under 30 tokens in length
min_token_length=36
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
   print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')


In [ ]:
pages_and_chunks_over_min_token_len=df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model=SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                    device="cpu")

#create a list of sentences to turn into numbers
sentences=[
    "The Sentences Transformers library provides an easy and open-source way to create embeddings.",
    "Sentence can be embedded one by one or as a list of strings.",
    "Embeddings are one of the most powerful concepts in machine learning!",
    "Learn to use embeddings well and you'll be well on your way to being an AI engineer."
]
#Sentences are encoded/embedded by calling model.encode()
embeddings=embedding_model.encode(sentences)
embeddings_dict=dict(zip(sentences,embeddings))

#see the embeddings
for sentence,embedding in embeddings_dict.items():
  print("Sentence:", sentence)
  print("Embedding:",embedding)
  print("")


In [ ]:
single_sentence="Yo! How cool are embeddings?"
single_embedding=embedding_model.encode(single_sentence)
print(f"Sentence:{single_sentence}")
print(f"Embedding:\n{single_embedding}")
print(f"Embedding size:{single_embedding.shape}")

In [ ]:
%%time
#Uncomment to see how long it takes to create embeddings on CPU
# # Make sure the model is on CPU
embedding_model.to("cpu")

# # Embed each chunk one by  one
for item in tqdm(pages_and_chunks_over_min_token_len):
  item["embedding"]=embedding_model.encode(item["sentence_chunk"])

In [ ]:
%%time
#send the model to the GPU
embedding_model.to("cuda")

#create embeddings one by one on the GPU
for item in tqdm(pages_and_chunks_over_min_token_len):
  item["embedding"]=embedding_model.encode(item["sentence_chunk"])

In [ ]:
#Turn text chunks into a single list.
text_chunks=[item['sentence_chunk'] for item in pages_and_chunks_over_min_token_len]

In [ ]:
%%time
#Embed all texts in batches
text_chunk_embeddings = embedding_model.encode(text_chunks,batch_size=32,convert_to_tensor=True)
text_chunk_embeddings

In [ ]:
#save embeddings to file
text_chunks_and_embeddings_df=pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path="text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path,index=False)

In [ ]:
#Import saved file and view
text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embedding_df_load.head()

In [ ]:
import random
import torch
import numpy as np
import pandas as pd

device="cuda" if torch.cuda.is_available() else "cpu"

#Imports texts and embedding df
text_chunks_and_embedding_df=pd.read_csv("text_chunks_and_embeddings_df.csv")

#convert embedding column back to np.array (it got converted to string when it got saved to csv)
text_chunks_and_embedding_df["embedding"]=text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"),))

#Convert texts and embedding df to list of dicts
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

#Convert embeddings to torch tensor and send to device (note: NumPy arrays are float64, torch tensors are float32 by default)
embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()),dtype=torch.float32).to(device)
embeddings.shape

In [ ]:
text_chunks_and_embedding_df.head()

In [ ]:
embeddings[0]

In [ ]:
from sentence_transformers import util, SentenceTransformer

embedding_model=SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                    device=device) #choose the device to load the model to

In [ ]:
# 1. Define the query
#Note: This could be anything. But since we're working with a nutrition textbook, we'll stick with nutrition-based queries.
query = "macronutrients functions"
print(f"Query:{query}")

#2. Embed the query to the same numerical space as the text examples
#Note: Its important to embed your query with the same model you embedded your examples with.
query_embedding = embedding_model.encode(query.convert_to_tensor=True)

#3 get similarity scores with the dot product (we'll time this for fun )

from time import perf_counter as timer

start_time =timer()
dot_scores=util.dot_score(a=query_embedding,b=embeddings)[0]
end_time=timer()

print(f"time take to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

#4 Get the top-k results (we'll keep this to 5)
top_results_dot_product=torch.topk(dot_scores,k=5)
top_results_dot_product


In [ ]:
larger_embeddings = torch.randn(100*embeddings.shape[0], 768).to(device)
print(f"Embeddings shape: {larger_embeddings.shape}")

#perform dot product across 168000 embeddings
start_time = timer()
dot_scores=util.dot_score(a=query_embedding,b=larger_embeddings)[0]
end_time = timer()
print(f"time take to get scores on {len(larger_embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

In [ ]:
#define helper function to print wrapped text
import textwrap

def print_wrapped(text, wrap_length=80):
  wrapped_text = textwrap.fill(text,wrap_length)
  print(wrapped_text)

In [ ]:
print(f"Query:'{query}'\n")
print("Results")
#Loop through zipped together scores and indices from torch.topk
for score, idx in zip(top_results_dot_product[0], top_results_dot_product[1]):
   print(f"score: {score:.4f}")
   #print relevant sentence chunk (since the scores are in descending order, the most relevant chunk will be first)
   print("Text:")
   print_wrapped(pages_and_chunks[idx]["sentence_chunk"])
   #print the page number too so we can reference the textbook further(and check the results)
   print(f"Page number: {pages and chunks[idx]['page_number']}")
   print("\n")

In [ ]:
import fitz
#Open PDF and load target page
pdf_path = "human-nutrition-text.pdf"#requires PDF to be downloaded
doc=fitz.open(pdf_path)
page = doc.load_page(5 + 41) #number of page (our doc starts page numbers on page 41)

#Get the image of the page
img=page.get_pixmap(dpi=300)

#Optional: save the image
#ima.save("output_filename.png")
doc.close()

#convert the Pixmap to a numpy array
img_array = np.frombuffer(img.samples_mv,dtype=np.uint8).reshape((img.h,img.w,img.n))

#Display the image using Matplotlib
import matplotlib.pyplot as plt
plt.figure(figsize=(13,10))
plt.imshow(img_array)
plt.title(f"Query: '{query}' | Most relevant page:")
plt.axis('off')
plt.show()



In [ ]:
import torch
def dot_product(vector1,vector2):
 return torch.dot(vector1,vector2)

def cosine_similarity(vector1,vector2):
  dot_product = torch.dot(vector1,vector2)
  #Get Eucledian/L2 norm of each vector (remove the magnitude, keeps direction)
  norm_vector1=torch.sqrt(torch.sum(vector1**2))
  norm_vector2=torch.sqrt(torch.sum(vector2**2))

  return dot_product / (norm_vector1 * norm_vector2)

#Example tensors
vector1=torch.tensor([1,2,3],dtype=torch.float32)
vector2=torch.tensor([1,2,3],dtype=torch.float32)
vector3=torch.tensor([4,5,6],dtype=torch.float32)
vector4=torch.tensor([-1,-2,-3],dtype=torch.float32)

#Calculate dot product
print("Dot product between vector1 and vector2:"), dot_product(vector1,vector2)
print("Dot product between vector1 and vector3:"), dot_product(vector1,vector3)
print("Dot product between vector1 and vector4:"), dot_product(vector1,vector4)

#calculate cosine similarity
print("Cosine similarity between vector1 and vector2:"), cosine_similarity(vector1,vector2)
print("Cosine similarity between vector1 and vector3:"), cosine_similarity(vector1,vector3)
print("Cosine similarity between vector1 and vector4:"), cosine_similarity(vector1,vector4)



In [ ]:
def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer=embedding_model,
                                n_resources_to_return: int=5,
                                print_time: bool=true)
"""
Embeds a query with model and returns top k scores and indices from embeddings.
"""
#Embeds the query
query_embedding = model.encode(query,convert_to_tensor=True)

#Get dot product scores on embeddings
start_time = timer()
dot_scores = util.dot_score(query_embedding,embeddings)[0]
end_time = timer()

if print_time:
  print(f"[INFO] Time taken to get scores on {len(embeddings)} embeddings:{end_time-start_time:.5f} seconds.")

scores, indices = torch.topk(input=dot_scores,k=n_resources_to_return)

return scores, indices
def print_top_results_and_scores(query: str,embeddings: torch.tensor,pages_and_chunks:list[dict]=pages_and_chunks,n_resources_to_return: int=5):

  """
  Takes a query, retrives most relevant resources
   and prints them out descending order.
   Note: Requires pages_and_chunks to be formatted in a specific way (see above for reference).
   """

   scores,indices = retrieve_relevant_resources(query=query,embeddings=embeddings,n_resources_to_return=n_resources_to_return)

   print(f"Query: '{query}'\n")
   print("Results")
   #Loop through zipped together scores and indicies
   for score,index in zip(scores,indices):
     print(f"Score:{score.4f}")
     #Print relevant sentence chunk (since the scores are in descending order,
     #the most relevant chunk will be first)
     print_wrapped(pages_and_chunks[index]["sentence_chunk"])
     #print the page number too so we can reference the textbook further and check the results
     print(f"page number:{pages_and_chunks[index]['page_number']}")
     print("\n")







Excellent Now let's test our fucntions out.

In [ ]:
query = "symptoms of pellagra"
#Get just the scores and indices of top related results
scores, indices = retrieve_relevant_resources(query=query,embeddings=embeddings)

scores,indices

In [ ]:
#Print out the texts of the top scores
print_top_results_and_scores(query=query,embeddings=embeddings)

In [ ]:
#Get GPU available memory
import torch
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb=round(gpu_memory_bytes/(2**30))
print(f"Available GPU memory:{gpu_memory_gb} GB")

In [ ]:
#Note: the following is Gemma focused, however, there are more and more LLMs of the 28 and 78 size appearing for local use.
if gpu_memory_gb < 5.1:
  print(f"Your available GPU memory is {gpu_memory_gb}GB, you may not have enough memory to run a Gemma Locally without quatization.")
elif gpu_memory_gb < 8.1:
  print(f"GPU memory:{gpu_memory_gb} | Recommended model: Gemma 2B in 4-bit precision.")
  use_quantization_config=True
  model_id="google/gemma-2b-it"
elif gpu_memory_gb < 19.0:
  print(f"GPU memory:{gpu_memory_gb} | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.")
  use_quantization_config=True
  model_id="google/gemma-2b-it"
elif gpu_memory_gb > 19.0:
  print(f"GPU memory:{gpu_memory_gb} | Recommend model: Gemma 7B in 4-bit of float16 precision.")
  use_quatization_config = False
  model_id="google/gemma-7b-it"
print(f"use_quatization_config set to: {use_quatization_config}")
print(f"model_id set to: {model_id}")

In [ ]:
from huggingface_hub import login
login(token="")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available

from transformers import BitsAndBytesConfig
quatization_config = BitsBytesConfig(load_in_4bit=True,bnb_4bit_compute_dtype=torch.float16)

attn_implementation = 'sdpa'

print(f"[INFO] Using attention implementation:{attn_implementation}")

model_id=model_id
print(f"[INFO] Using model_id: {model_id}")

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)

llm_model=AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id,
                                               torch_dtype=torch.float16,#datatype to use, we want float16
                                               quantization_config=quatization_config if use_quatization_config else None,
                                               low_cpu_mem_usage=False,#Use full memory
                                               attn_implementation=attn_implementation)#which attention version to use


if not use_quatization_config: #Quatization takes care fo device setting automatically, so if its not used , send model to Gpu
  llm_model.to(device="cuda")



In [ ]:
llm_model

In [ ]:
def get_model_num_params(model:torch.nn.module):
  return sum([param.numel() for param in model.parameters()])
get_model_num_params(llm_model)

In [ ]:
def get_model_mem_size(model: torch.nn.Module):
  """
   get how much memory a Pytorch model takes up.
   """
   #Get model parameters and buffer sizes
   mem_params=sum([param.nelement() * param.element_size() for param in model.parameters()])
   mem_buffers = sum([buf.nelement() * buf.element_size() for buf in model.buffers()])

   #calculate various model sizes
   model_mem_bytes = mem_params + mem_buffers #in bytes
   model_mem_mb=model_mem_bytes / (1024**2)# in megabytes
   model_mem_gb = model_mem_bytes / (1024**3) # in gigabytes

   return {"model_mem_bytes": model_mem_bytes,
           "model_mem_mb":round(model_mem_mb,2),
           "model_mem_gb":round(model_mem_gb,2)}

get_model_mem_size(llm_model)

In [ ]:
input_text="What are the macronutrients, and what roles do they paly in the human body?"
print(f"Input text:\n{input_text}")

#create prompt template for instruction-tuned model
dialogue_template=[
    {"role":"user",
     "content":input_text}
]
#Apply the chat template
prompt=tokenizer.apply_chat_template(conversation=dialogue_template,
                                     tokenize=False,#Keep as raw text(not tokenized)
                                     add_generation_prompt=True)
print(f"\nPrompt (formatted):\n{prompt}")

In [ ]:
%%time
#Tokenize the input text (turn it into numbers) and send it to GPU
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
print(f"Model input (tokenized):\n{input_ids}\n")

outputs=llm_model.generate(**input_ids,
                           max_new_tokens=256) #define the maximum number of new tokens to create
print(f"Model output (tokens):\n{outputs[0]}\n")

In [ ]:
#Decode the output tokens to text
outputs_decoded = tokeizer.decode(outputs[0])
print(f"Model output (decoded):\n{outputs_decoded}\n")

In [ ]:
print(f"Input text:{input_text}\n")
print(f"Output text:\n{outputs_decoded.replace(prompt,'').replace('<bos>','').replace('<eos>','')}")

In [ ]:
#NUtrition-style questions generated with GPT4
gpt4_questions=[
    "what are the macronutrients, and what roles do they play in the human body?",
    "How do vitamins and minerals differ in their roles and importance for health?",
    "Describe the process of digestion and absorption of nutrients in the human body.",
    "What role does fibre play in digestion? Name five fibre conataining foods.",
    "Explain the concept of energy balance and its importance in weight management."

]

#Manually craeted question list
manual_questions=[
    "How often should infants be breastfed?".
    "What are symptoms of pellagra?",
    "How does salvia help with digestion?",
    "What is the RDI for protein per day?",
    "Water soluble vitamins"]
query_list = gpt4_questions + manual_questions

In [ ]:
import random
query = random.choice(query_list)

print(f"Query: {query}")

#Get just scores and indices of top related results
scores,indices = retrieve_relevant_resources(query=query,embeddings=embeddings)

scores , indices

In [ ]:
def prompt_formatter(query: str,
                     context_items: list[dict]) -> str:
    """
    Augments query with text-based context from context_items.
    """
    #join context items into one dotted paragraph
    context = "-" + "\n-".join([item["sentence_chunk"] for item in context_items])

    #create a base prompt with examples to help the model
    #Note:This is very customizable,'ve chosen to use 3 examples of the answer style we'd like.



   I
    # We could also write this in a txt file and import it in if we wanted.
    base_prompt = """Based on the following context items, please answer the query.
    Give yourself room to think by extracting relevant passages from the context before answering the query.
    Don't return the thinking, only return the answer.
    Make sure your answers are as explanatory as possible.
    Use the following examples as reference for the ideal answer style.

    Example 1:
    Query: What are the fat-soluble vitamins?
    Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and are stored in the body's fatty tissues and liver. Because they can be stored for long periods, they do not need to be consumed every day.

    Example 2:
    Query: What are the causes of type 2 diabetes?
    Answer: Type 2 diabetes is primarily caused by a combination of genetic and lifestyle factors. It is often associated with overnutrition and obesity, which can lead to insulin resistance—a condition where the body's cells do not respond effectively to insulin. Other major risk factors include a sedentary lifestyle and a family history of the disease.

    Example 3:
    Query: What is the importance of hydration for physical performance?
    Answer: Hydration is crucial for physical performance because water plays key roles in maintaining blood volume, regulating body temperature through sweating, and lubricating joints. Proper hydration ensures that muscles receive oxygen, nutrients are transported efficiently, and it helps prevent fatigue, muscle cramps, and dizziness during exercise.

    Now use the following context items to answer the user query:
    Context: {context}
    Relevant passages: <extract relevant passages from the context here>
    User query: {query}
    Answer:"""
           #Update base prompt with context items and query
           base_prompt=base_prompt.format(context=context,query=query)

           #Create prompt template for instruction-tuned model
           dialogue_template=[
               {"role":"user",
                "content":base_prompt}
           ]

           #Apply the chat template
           prompt=tokenizer.apply_chat_template(conversation=dialogue_template,
                                                tokenize=False,
                                                add_generation_prompt=True)
           return prompt

In [ ]:
query=random.choice(query_list)
print(f"Query:{query}")

#Get relevant resources
scores,indices = retriev_relevant_resources(query=query,embeddings=embeddings)

#Create a list of context items
context_items=[pages_and_chunks[i] for i in indices]

#Format prompt with context items
prompt=prompt_formatter(query=query,context_items=context_items)

print(prompt)

In [ ]:
%%time
input_ids = tokenizer(prompt,return_tensors="pt").to("cuda")

#generate an output of tokens
outputs = llm_model.generate(**input_ids,temprature=0.7,do_sample=True,max_new_tokens=256)

#Turn the output tokens into text
output_text = tokenizer.decode(outputs[0])

print(f"Query: {query}")
print(f"RAG answer:\n{output_text.replace(prompt,'')}")

In [ ]:
def ask(query,temprature=0.7,max_new_tokens=512,format_answer_text=true,return_answer_only=True):
  """
  Takes a query, finds relevant resources/context and generates an answer to the query based on the relevant resources.
  """
  #Get just the scores and indices of top related results
  scores,indices=retrieve_relevant_resources(query=query,embeddings=embeddings)

  #Create a list of context items
  context_items=[pages_and_chunks[i] for i in indices]

  #Add score to context item
  for i,item in enumerate(context_items):
    item["score"]=scores[i].cpu() #return score back to CPU

  #Format the prompt with context items
  prompt=prompt_formatter(query=query,context_items=context_items)
  #Tokenize the prompt
  input_ids=tokenizer(prompt,return_tensors="pt").to("cuda")

  #Generate an output of tokens
  outputs=llm_model.generate(**input_ids,temperature=temperature,do_sample=True,max_new_tokens=max_new_tokens)

  #Turn the output tokens into text
  output_text=tokenizer.decode(outputs[0])

  if format_answer_text:
    #Replace special tokens and unnecessary help message

    output_text=output_text.replace(prompt,'').replace("<bos>","").replace("<eos>","").replace("Sure,here is the answer to the user query:\n\n","")
  #only return the answer without the context items
  if return_answer_only:
    return output_text
  return  output_text, context_items

In [ ]:
query=random.choice(query_list)
print(f"Query:{query}")

#Answer query with context and return context
answer,context_items=ask(query=query,temperature=0.7,max_new_tokens=512,return_answer_only=False)

print(f"Answer:\n")
print_wrapped(answer)
print(f"Context items:")
context_items

Evaluation of our results

In [ ]:
!pip install -q rags datasets

In [ ]:
#Install required packages
#!pip install raga datasets

import pandas as pd
from datasets import dataset
from raga import evaluate
from ragas.metrics import(
    context_precision,
    context_recall,
    answer_relevancy,
    faithfulness
)

#Try to import additional metrics if available
try:
   from raga.metrics import context_entity_recall
except ImportError:
  context_entity_recall = None
  print("Context_entity_recall not available in this version")

try:
  from raga.metrics import noise_robustness
except ImportError:
  noise_robustness = None
  print("moise_robustness not available in this version")

In [ ]:
#Define evaluation question in this version

eval_questions=[
    "How often should infants be breastfed?",
    "What are symptoms of pellagra?",
    "How does salvia help with digestion?",
    "What is the RDI for protein per day, based on your weight?",
    "What are micronutrients?"
]

ground_truth_answers=[
    "Breastfeed infants on demand, about 8 to 12 times daily.",
    "Pellagra's symptoms are dermatitis, diarrhea, dementia, and ultimately death.",
    "Saliva uses enzymes to start breaking down fats and starches.",
    "The RDI is 0.8 grams of protein per kilogram bodyweight.",
    "Micronutrients are essential vitamins and minerals needed in small amounts."
    ]

In [ ]:
def generate_rag_answer(query):
  """Generate RAG answer for a given query"""
  #Get relevant resources
  scores,indices=retrieve_relevant_resources(query=query,embeddings=embeddings)

  #Create a list of context items
  context_items=[pages_and_chunks[i] for i in indices]